In [44]:
import pandas as pd 
import numpy as np
import gc
import matplotlib.pyplot as plt

In [ ]:
#!pip install openpyxl

# Первичный анализ данных

1. Первичный анализ данных \
1.1 Проверить уникальность id в полисах и убытках \
1.2 Подтянуть рисковые факторы и характеристики от полисов к убыткам \
1.3 Провести агрегацию выплат до убытков \
1.4 Подтянуть количество и сумму убытков к каждому полису \
1.5 Притянуть к полисам и убыткам – количество водителей, возраст самого молодого, стаж водителя с самым маленьким стажем. \
1.6 Провести проверки данных. Это творческое задание. Примеры проверок – дата начала полиса больше даты окончания полиса,дата убытка между датой начала и окончания полиса и т.д. \
1.7 Построить распределения количества убытков и размера убытка – посмотреть средние, минимальные и максимальные значения, перцентили. \
1.8 Рассчитать экспозицию и заработанную премию для календарного анализа \
1.9 Отдельно для календарного и андеррайтингово подходов (для убытков - тяжесть и полисов - частота) посмотреть по каждому фактору: \
1.9.1 заполненность факторов, построить распределения значений фактора (для каждого фактора) в портфеле (по таблице с полисами) \
1.9.2 построить графики количества убытков/тяжести убытков в зависимости от значений фактора, для каждого уровня фактора оценить дисперсию количества и тяжести; \
1.9.3 проанализировать устойчивость одномерных распределений по годам (убытка, заключения полиса)


### 1.1 Проверить уникальность id в полисах и убытках

In [2]:
df_claim = pd.read_excel('/workspaces/BDAktuarki/data/claim_reestr.xlsx')
df_polis = pd.read_excel('/workspaces/BDAktuarki/data/polis_reestr.xlsx')
df_drivers = pd.read_excel('/workspaces/BDAktuarki/data/drivers_reestr.xlsx')

print(f'Процент уникальных id полисов в реестре выплат: {df_claim["POLIS_ID"].nunique() / len(df_claim) * 100}')
print(f'Процент уникальных id полисов в реестре полисов: {df_polis["POLIS_ID"].nunique() / len(df_polis) * 100}')
print(f'Процент уникальных id полисов в реестре водителей: {df_drivers["POLIS_ID"].nunique() / len(df_drivers) * 100}')
print(f'Процент уникальных id убытков: {df_claim["PAY_ID"].nunique() / len(df_claim) * 100}')

Процент уникальных id полисов в реестре выплат: 76.55517854257555
Процент уникальных id полисов в реестре полисов: 100.0
Процент уникальных id полисов в реестре водителей: 58.260494340397095
Процент уникальных id убытков: 100.0


### 1.6 Провести проверки данных. 
Это творческое задание. Примеры проверок – дата начала полиса больше даты окончания полиса, дата убытка между датой начала и окончания полиса и т.д.

In [3]:
# Первичное преобразование типов для непустых признаков дат
for col in df_polis.columns:
    if ('_DT' in col) and (df_polis[col].isna().sum() == 0):
        print(col)
        df_polis[col] = pd.to_datetime(df_polis[col]) 

for col in df_claim.columns:
    if ('_DT' in col) and (df_claim[col].isna().sum() == 0):
        print(col)
        df_claim[col] = pd.to_datetime(df_claim[col], dayfirst=True)

CREATE_DT
START_DT
END_DT
DTP_DT
CLAIM_DT
PAY_DT


In [45]:
df_polis_to_j = df_polis.set_index('POLIS_ID').copy()
df_claim_to_j = df_claim.set_index('POLIS_ID').copy()
df_j_pls_clm = df_polis_to_j.join(df_claim_to_j, how='left').reset_index()
df_j_pls_clm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118574 entries, 0 to 118573
Data columns (total 37 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   POLIS_ID         118574 non-null  int64         
 1   CREATE_DT        118574 non-null  datetime64[ns]
 2   START_DT         118574 non-null  datetime64[ns]
 3   END_DT           118574 non-null  datetime64[ns]
 4   PREMIUM          118573 non-null  float64       
 5   DISSOLUTION_DT   0 non-null       float64       
 6   VIN              108264 non-null  object        
 7   CAR_ISSUE        118176 non-null  float64       
 8   CAR_DIAG_DT      78205 non-null   object        
 9   PERS_POLIS_OWN   118501 non-null  object        
 10  PERS_CAR_OWN     118574 non-null  object        
 11  KLADR_CODE       118164 non-null  float64       
 12  MULTIDRIVE_FLG   118574 non-null  int64         
 13  BONUS_MALUS      118556 non-null  float64       
 14  CAR_ENG_PWR      118

In [5]:
del df_polis_to_j, df_claim_to_j
gc.collect()

122

In [34]:
# Дата начала полиса больше даты окончания полиса
mask = df_polis['START_DT'] > df_polis['END_DT']
assert len(df_polis[mask]) == 0

# Дата убытка между датой начала и окончания полиса
mask_on_pay = ~df_j_pls_clm['PAY_ID'].isna()
mask_on_date_lb = df_j_pls_clm['PAY_DT'] >= df_j_pls_clm['START_DT']
mask_on_date_lb = df_j_pls_clm['PAY_DT'] <= df_j_pls_clm['END_DT']
print('Процент наблюдений среди случаев с ДТП в которых дата убытка между датой начала и окончания', end=' ')
print(len(df_j_pls_clm[mask_on_pay & mask_on_date_lb & mask_on_date_lb]) / len(df_j_pls_clm[mask_on_pay]) * 100)

Процент наблюдений среди случаев с ДТП в которых дата убытка между датой начала и окончания 79.30198739699466


In [42]:
# Проверка численных признаков на равенство положительную определенность 
num_feats_polis = ['PREMIUM', 'BONUS_MALUS', 'CAR_ENG_PWR', 'CAR_WEIGHT', 'CAR_SEATS_CNT']
num_feats_drivers = ['DRIVER_AGE', 'DRIVER_EXP', 'BONUS_MALUS']
num_feats_claim = ['PAY_AMT']
num_feats_all = [num_feats_polis, num_feats_drivers, num_feats_claim]
all_dfs = {'polis_reestr': df_polis,
           'drivers_reestr': df_drivers,
           'claim_reestr': df_claim}

print('Кол-во наблюдений в каждом численном признаке равных 0')
for feat_list, df_key in zip(num_feats_all, all_dfs.keys()):
    df = all_dfs[df_key].copy()
    lens_wrong = [len(df[df[feat] == 0]) for feat in feat_list]
    print(df_key, end=': ')
    print(*zip(feat_list, lens_wrong))
print()
print('Кол-во наблюдений в каждом численном признаке меньше 0')
for feat_list, df_key in zip(num_feats_all, all_dfs.keys()):
    df = all_dfs[df_key].copy()
    lens_wrong = [len(df[df[feat] < 0]) for feat in feat_list]
    print(df_key, end=': ')
    print(*zip(feat_list, lens_wrong))


Кол-во наблюдений в каждом численном признаке равных 0
polis_reestr: ('PREMIUM', 75) ('BONUS_MALUS', 0) ('CAR_ENG_PWR', 1) ('CAR_WEIGHT', 303) ('CAR_SEATS_CNT', 331)
drivers_reestr: ('DRIVER_AGE', 0) ('DRIVER_EXP', 1131) ('BONUS_MALUS', 0)
claim_reestr: ('PAY_AMT', 1)

Кол-во наблюдений в каждом численном признаке меньше 0
polis_reestr: ('PREMIUM', 1) ('BONUS_MALUS', 0) ('CAR_ENG_PWR', 0) ('CAR_WEIGHT', 0) ('CAR_SEATS_CNT', 0)
drivers_reestr: ('DRIVER_AGE', 0) ('DRIVER_EXP', 2) ('BONUS_MALUS', 0)
claim_reestr: ('PAY_AMT', 0)


In [43]:
del all_dfs, df
gc.collect()

1016

### 1.7 Построить распределения количества убытков и размера убытка 
Посмотреть средние, минимальные и максимальные значения, перцентили.